# Gerando conjunto de treinamentos

In [1]:
import rasterio
from rasterio import features
import geopandas as gpd
import pandas as pd
import numpy as np
from shapely.geometry import box
import glob
import re

In [2]:
scm_licoes = pd.read_csv('downloads/imagens/SCMs-licoes.csv')
licoes_utilizadas = scm_licoes.loc[scm_licoes["presente"] == 1, 'scm']
licoes = gpd.read_file('licoes/licoes-por-scm.gpkg')
licoes = licoes[licoes.scm.isin(licoes_utilizadas)]
gdf_quadras = gpd.read_file('zip://downloads/SIRGAS_SHP_quadraMDSF.zip!/SIRGAS_SHP_quadraMDSF/SIRGAS_SHP_quadraMDSF.shp')

In [3]:
# TODO - Colocar esse parâmetros em um arquivo para importacao
tamanho_janela = 24 # tamanho em pixels
sobreposicao_de_janelas = 18 # sobreposição das janelas em pixels

In [4]:
for f in glob.glob("pre-process/grayscales/*.jp2"):
    
    data, target = [], []

    dataset = rasterio.open(f)
    scm = re.search('gray-scale-(.*).jp2', f).group(1)
    
    grayscale = dataset.read(1).astype('float32')

    labels = features.rasterize(
        ((g, 255) for g in licoes[licoes.scm == scm].geometry.explode()),
        out_shape=dataset.shape,
        transform=dataset.transform)

    quadras_scm = gpd.clip(gdf_quadras, gpd.GeoSeries(box(*dataset.bounds))).unary_union

    quadras = features.rasterize(
        ((g, 255) for g in quadras_scm),
        out_shape=dataset.shape,
        transform=dataset.transform)

    sobel_image = np.load(f'pre-process/sobel/sobel-{scm}.npy').astype('float32')
    lbp = np.load(f'pre-process/sobel/sobel-{scm}.npy').astype('float32')

    rua = 0
    faixa = 0

    for i in np.arange(0, grayscale.shape[0] - tamanho_janela, tamanho_janela - sobreposicao_de_janelas):
        for j in np.arange(0, grayscale.shape[1] - tamanho_janela, tamanho_janela - sobreposicao_de_janelas):
            # Se mais de 98% dos pixels estiverem fora da quadra
            if np.count_nonzero(quadras[i:i+tamanho_janela, j:j+tamanho_janela] == 255) < 0.02 * (tamanho_janela ** 2):
                # print(i, j)
                img_gray = grayscale[i:i+tamanho_janela, j:j+tamanho_janela]
                img_sobel = sobel_image[i:i+tamanho_janela, j:j+tamanho_janela]
                img_lbp = lbp[i:i+tamanho_janela, j:j+tamanho_janela]

                # Data Augmentation
                
                img = np.concatenate([
                    img_gray,
                    img_sobel,
                    img_lbp
                ]).reshape(1, 3 * (tamanho_janela ** 2))
                
                ## Caso mais de 66% dos pixels estiverem dentro da lição (faixa)
                if np.count_nonzero(labels[i:i+tamanho_janela, j:j+tamanho_janela] == 255) > 0.66 * (tamanho_janela ** 2):
                    faixa += 1
                    data.append(img.reshape(1, 3 * (tamanho_janela ** 2)))
                    target.append(1)

                else:
                    # adiciona lição negativa somente se for múltiplo do tamanho da janela
                    # estratégia para reduzir o número de licoes negativas
                    if i % tamanho_janela == 0 and j % tamanho_janela == 0:
                        rua += 1
                        data.append(img.reshape(1, 3 * (tamanho_janela ** 2)))
                        target.append(0)

    np.save(f'pre-process/data/data-{scm}.npy', np.array(data).squeeze())
    np.save(f'pre-process/data/target-{scm}.npy', np.array(target))
    print(f"SCM: {scm}: {rua} ruas e {faixa} faixas")
    # break


SCM: 3313-412: 5909 ruas e 595 faixas
SCM: 3325-164: 8359 ruas e 500 faixas
SCM: 3326-461: 5756 ruas e 534 faixas
SCM: 3313-432: 7907 ruas e 2809 faixas
SCM: 3315-211: 6940 ruas e 513 faixas
SCM: 3325-332: 5595 ruas e 636 faixas
SCM: 3326-424: 6324 ruas e 391 faixas
SCM: 3314-311: 7292 ruas e 2104 faixas
SCM: 3315-132: 8665 ruas e 1372 faixas
SCM: 3313-414: 8706 ruas e 1912 faixas
SCM: 3325-334: 5223 ruas e 1237 faixas
SCM: 3316-221: 5980 ruas e 1764 faixas


In [5]:
np.arange(0, 480, 24-16) % 24 == 0

array([ True, False, False,  True, False, False,  True, False, False,
        True, False, False,  True, False, False,  True, False, False,
        True, False, False,  True, False, False,  True, False, False,
        True, False, False,  True, False, False,  True, False, False,
        True, False, False,  True, False, False,  True, False, False,
        True, False, False,  True, False, False,  True, False, False,
        True, False, False,  True, False, False])

In [6]:
np.arange(0, 480, 24-16)

array([  0,   8,  16,  24,  32,  40,  48,  56,  64,  72,  80,  88,  96,
       104, 112, 120, 128, 136, 144, 152, 160, 168, 176, 184, 192, 200,
       208, 216, 224, 232, 240, 248, 256, 264, 272, 280, 288, 296, 304,
       312, 320, 328, 336, 344, 352, 360, 368, 376, 384, 392, 400, 408,
       416, 424, 432, 440, 448, 456, 464, 472])

In [7]:
np.arange(9).reshape(3, 3)

array([[0, 1, 2],
       [3, 4, 5],
       [6, 7, 8]])

In [8]:
for k in range(4):
    print(np.rot90(np.arange(9).reshape(1, 3, 3), k=k, axes=(1,2)))


[[[0 1 2]
  [3 4 5]
  [6 7 8]]]
[[[2 5 8]
  [1 4 7]
  [0 3 6]]]
[[[8 7 6]
  [5 4 3]
  [2 1 0]]]
[[[6 3 0]
  [7 4 1]
  [8 5 2]]]


In [9]:
np.flip(np.arange(9).reshape(1, 3, 3), axis=1)

array([[[6, 7, 8],
        [3, 4, 5],
        [0, 1, 2]]])